In [1]:
# 将路网拓扑上的节点与实际卡口设备进行匹配 2025/01/15 by chenxinyi
# 卡口数据更新：2023年4月 5831个卡口

import geopandas as gpd
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# 小数点后默认保留7位数字
pd.options.display.precision = 7

In [2]:
def closest_point(target_point, points, threshod):
    # res_point = min(points, key=lambda x: geodesic((x[1], x[0]), (target_point[0][1], target_point[0][0])).m)
    # l = geodesic((res_point[1], res_point[0]), (target_point[0][1], target_point[0][0])).m
    # 计算两个经纬度之间的测地距离，lat在前，lon在后
    l = []
    for x in points:
        value = geodesic((x[1], x[0]), (target_point[0][1], target_point[0][0])).m
        l.append(value)
    
    if min(l) > threshod:
        return [], []
    
    zipped1 = zip(points, l)
    sort_zipped = sorted(zipped1, key=lambda x: (x[1], x[0]))  # 排序
    zipped2 = zip(*sort_zipped)
    sort_points, sort_l = [list(x) for x in zipped2]  # 拆分元组
    return sort_points, sort_l

In [3]:
# 读取实际卡口位置
shp_file = r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\佛山行政区和路网数据\中心城区路网和卡口\中心城区卡口点位.shp'
bayonets = gpd.read_file(shp_file)
bayonets.head(3)

,KKBH,KKMC,RoadName,Longitude,Latitude,geometry
0,440604700121006001,佛山市禅城区东平路佛山环境监测站路段（南往北）,东平路,113.032155,23.023915,POINT (113.03216 23.02392)
1,440604700121006000,佛山市禅城区东平路佛山环境监测站路段（北往南）,东平路,113.032304,23.023730,POINT (113.0323 23.02373)
2,440604679751003002,佛山市禅城区张槎西路季华北路段（东往西）,季华北路,113.036078,23.030748,POINT (113.03608 23.03075)


In [4]:
# 读取路网节点
pos = np.load('center_pos.npy', allow_pickle=True).item()
pos_name = list(pos.keys())
pos_coords = list(pos.values())

# 构建dataframe
pos_df = pd.DataFrame({'pos_name': pos_name, 'pos_coords': pos_coords})
pos_df['lat'] = ''
pos_df['lon'] = ''
for i in range(0, len(pos_df)):
    pos_df.loc[i, 'lat'] = pos_df.loc[i, 'pos_coords'][1]
    pos_df.loc[i, 'lon'] = pos_df.loc[i, 'pos_coords'][0]
    
pos_df.head(3)

,pos_name,pos_coords,lat,lon
0,O-39092939,"(113.1032872, 23.0373638)",23.0373638,113.1032872
1,2-39092939,"(113.1033508, 23.0379217)",23.0379217,113.1033508
2,3-39092939,"(113.1033662, 23.0380568)",23.0380568,113.1033662


In [5]:
# 最小距离 m
threshod = 60

# 路网节点与真实卡口匹配表 pos_name,KKBH,dist
pos_bayonet = [[], [], []]

for sy, bayonet in bayonets.iterrows():
    bayonet_kkbh = bayonet['KKBH']
    bayonet_coord = bayonet['geometry'].coords[:]
    # print(bayonet_coord)
    
    # 提取卡口50m范围内的路网节点
    # 23°纬度圈周长：40030173 * cos23° = 36847968 m
    # 经度（东西方向）1米实际度（23°纬度为例）：360°/36847968m = 9.76987387744e-6 = 0.00000977°
    # 纬度（南北方向）1米实际度：360°/40030173m = 8.993216192195822e-6 = 0.00000899°
    lat_maxrange = bayonet_coord[0][1] + 0.00000899 * threshod
    lat_minrange = bayonet_coord[0][1] - 0.00000899 * threshod
    lon_maxrange = bayonet_coord[0][0] + 0.00000977 * threshod
    lon_minrange = bayonet_coord[0][0] - 0.00000977 * threshod
    subset_pos_df = pos_df[(pos_df['lat'] >= lat_minrange) & (pos_df['lat'] <= lat_maxrange) & 
                           (pos_df['lon'] >= lon_minrange) & (pos_df['lon'] <= lon_maxrange)]
    
    if subset_pos_df.empty:
        print(f"{sy}-{bayonet['KKBH']} min_dist > {threshod}m ×")
        continue
    
    # 计算卡口与附近路网节点的距离
    subset_pos_coords = subset_pos_df['pos_coords'].tolist()
    sort_coord, sort_dist = closest_point(bayonet_coord, subset_pos_coords, threshod)  # 排序
    # print(sort_dist[:5])
    
    if sort_coord == []:
        print(f"{sy}-{bayonet['KKBH']} min_dist > {threshod}m ×")
        continue
    
    # 判断距离卡口最近的路网节点，并与之匹配
    for i in range(len(sort_coord)):
        coord = sort_coord[i]
        dist = round(sort_dist[i], 2)
        index = pos_coords.index(coord)
        match_pos = pos_name[index]    

        if match_pos not in pos_bayonet[0] and dist <= threshod:  
            min_pos = pos_name[index]
            pos_bayonet[0].append(min_pos)
            pos_bayonet[1].append(bayonet_kkbh)
            pos_bayonet[2].append(dist)
            print(f"{sy}-{bayonet['KKBH']} min_dist = {dist}m √")
            break
            
print(1)

0-440604700121006001 min_dist > 60m ×
1-440604700121006000 min_dist = 46.32m √
2-440604679751003002 min_dist = 36.0m √
3-440604679751003003 min_dist = 36.44m √
4-440604660101003001 min_dist = 12.35m √
5-440604660101002000 min_dist = 49.39m √
6-440604660101002001 min_dist = 22.15m √
7-440604600081005001 min_dist = 30.41m √
8-440604600081005002 min_dist = 37.81m √
9-440604660101003000 min_dist = 22.33m √
10-440604760451001000 min_dist > 60m ×
11-440604760451001001 min_dist = 6.93m √
12-440604305060005701 min_dist = 26.44m √
13-440604305060005700 min_dist = 26.7m √
14-440604305060005702 min_dist = 18.86m √
15-440604600191001006 min_dist = 49.19m √
16-440604660081005004 min_dist > 60m ×
17-440604305060005703 min_dist = 40.73m √
18-440604305050005801 min_dist = 43.06m √
19-440604660081005002 min_dist > 60m ×
20-440604600191001001 min_dist = 22.17m √
21-440604305060005201 min_dist = 9.54m √
22-440604760151006002 min_dist = 12.52m √
23-440604700121001002 min_dist = 28.44m √
24-440604700121001

In [6]:
# 打印结果
print("实际卡口设备数:", len(bayonets))
print('路网拓扑结构中的路网节点数量:', len(pos_df))
print("路网节点匹配成功率: {:.2%}".format(len(pos_bayonet[0]) / len(bayonets)))
print("路网真实卡口节点数:", len(pos_bayonet[0]), '占比: {:.2%}'.format(len(pos_bayonet[0]) / len(pos_df)))
print("路网虚拟卡口节点数:", len(pos_df) - len(pos_bayonet[0]), '占比: {:.2%}'.format((len(pos_df) - len(pos_bayonet[0])) / len(pos_df)))

实际卡口设备数: 923
路网拓扑结构中的路网节点数量: 19609
路网节点匹配成功率: 79.74%
路网真实卡口节点数: 736 占比: 3.75%
路网虚拟卡口节点数: 18873 占比: 96.25%


In [10]:
# 保存路网节点与实际卡口编号匹配表
np.save("center_realpos.npy", pos_bayonet)
result = pd.DataFrame({'node': pos_bayonet[0], 'KKBH': pos_bayonet[1], 'dist_m': pos_bayonet[2]})
result.to_csv("center_realpos.csv", index=False)